**#1  Express the map-coloring problem from R&N Figure 6.1 as a CSP, and solve it.**

Make the colors "B", "G", "R", "Y"

How many solutions are there using 
* two colors
* three colors
* four colors

In [ ]:
from constraint import *
colors = ["B", "G", "R", "Y"]
states = ["WA", "NT", "SA", "Q", "NSW", "V", "T"]
adjacencies = [("WA", "NT"), ("WA", "SA"), ("NT", "SA"), ("NT", "Q"), ("Q", "SA"),\
             ("Q", "NSW"), ("SA", "NSW"), ("SA", "V"), ("NSW", "V")]

def solveFor(nc=3):
    problem= Problem()
    problem.addVariables(states, colors[0:nc])
    return problem.getSolutions()

In [ ]:
print(solveFor(3)[0])

In [ ]:
print(len(solveFor(2)))
print(len(solveFor(3)))
print(len(solveFor(4)))

----------------------------------------------

**#2 Solve the classic cryptarithmetic problem**

<pre>
    S E N D
 +  M O R E
 -------------
  M O N E Y
</pre>

where each letter is assigned a digit.  There can be no leading zeros, 
so S and M cannot be 0

R&N Figure 6.2 should be helpful.

Notice in Figure 6.2 there is an "all different" constraint on the values of the letters.

Implement that constraint.

What does that additional constraint do to the number of solutions?

===============================
To discuss in lab

1. The general approach, column by column
1. Demonstrate for two columns
1. The general solution

In [ ]:
from constraint import *
problem = Problem()
problem.addVariables(["D", "E", "Y"], [0,1,2,3,4,5,6,7,8,9])
problem.addVariables(["N", "R"], [0,1,2,3,4,5,6,7,8,9])
problem.addVariables(["C0"], [0])
problem.addVariables(["C1"], [0,1])
problem.addVariables(["C2"], [0,1])

# Column #1
problem.addConstraint(lambda x, y, z, c1: (x + y + c1)%10 == z, ["D", "E", "Y", "C0"])
# carry out
problem.addConstraint(lambda x, y, c1, c2: int((x + y + c1)/10) == c2, ["D", "E", "C0", "C1"])

### Column #2
# carry in and Y
problem.addConstraint(lambda x, y, z, c1: (x + y + c1)%10 == z, ["N", "R", "E", "C1"])
# carry out
problem.addConstraint(lambda x, y, c1, c2: int((x + y + c1)/10) == c2, ["N", "R", "C1", "C2"])

#######################
solutions = problem.getSolutions()
print(solutions)

In [ ]:
########################################
#  This is the more general-ish solution
#
#    S E N D
# +  M O R E
# -------------
#  M O N E Y
#   c3
#     c2
#       c1
#         c0

from constraint import *

leadingLetters = ["S", "M"]
letters = ["N", "D", "O", "R", "E", "Y"]
carries = ["C0", "C1", "C2", "C3"]
problem = Problem()

problem.addVariables(leadingLetters, range(1,10))
problem.addVariables(letters, range(0,10))
problem.addVariables(carries, range(0,2))

def addAdditionConstraint(a1, a2, sum, incarry, outcarry):
        problem.addConstraint(lambda a1, a2, sum, incarry: sum == ((a1+a2+incarry) % 10), [a1, a2, sum, incarry])
        problem.addConstraint(lambda a1, a2, incarry, outcarry: outcarry == int((a1+a2+incarry)/10),\
                              [a1, a2, incarry, outcarry])

def addInequalities():
    for l1 in leadingLetters + letters:
        for l2 in leadingLetters + letters:
            if (l1 != l2):
                problem.addConstraint(lambda a1, a2: a1 != a2, [l1, l2])

problem.addConstraint(lambda c: c == 0, ["C0"])    
addAdditionConstraint("D", "E", "Y", "C0", "C1")
addAdditionConstraint("N", "R", "E", "C1", "C2")
addAdditionConstraint("E", "O", "N", "C2", "C3")
addAdditionConstraint("S", "M", "O", "C3", "M")
addInequalities()
                                      
print(len(problem.getSolutions()))
print(problem.getSolutions())

**#3 Solve the "slightly bigger" job shop example from the Moore tutorial, slide 44**

Short summary

1. There are four jobs 1..4;  each job has between two and three subtasks
1. For each jobs, the subtasks must be done in sequence
1. Each subtask requires a resource,  and two subtasks that use the same resource cannot be scheduled at the same time
1. All subtasks take three time units to complete
1. All subtasks are ready for execution at time 0, and must complete at or before time 15

From the jobs/subtasks/resource uses on the diagram, assign a start time to each subtask that satisfies all the constraints above.




In [ ]:
#Job 1
#   (o11, r1) (o12, r2) (o13, r3)
#Job 2
#   (o21, r1)  (o22, r2)
#Job 3
#   (o31, r3) (o32, r1) (o33, r2)
#Job 4
#   (o41, r4) (o42, r2)

example = [[("o11", "r1"), ("o12", "r2"), ("o13", "r3")],
           [("o21", "r1"), ("o22", "r2")],
           [("o31", "r3"), ("o32", "r1"), ("o33", "r2")],
           [("o41", "r4"), ("o42", "r2")]]

# Partition 0..15 into five buckets, 0..2, 3..5, 6..8, 9..11, 12..14
# A task fits exactly into one of these, and no benefit to starting
# a task other than at a boundary
from constraint import *

def addDisjointConstraint(problem, t1, t2):
    problem.addConstraint(lambda t1, t2: t1 != t2, [t1, t2])

def addOrderConstraint(problem, t1, t2):
    problem.addConstraint(lambda t1, t2: t1<t2, [t1, t2])

##############################################
tasks = ["o11", "o12", "o13", "o21", "o22", "o31", "o32", "o33", "o41", "o42"]

problem = Problem()
problem.addVariables(tasks, range(0, 5))

# R1 shared by o11, o21, o32

addDisjointConstraint(problem, "o11", "o21")
addDisjointConstraint(problem, "o21", "o32")
addDisjointConstraint(problem, "o11", "o32")

# R2 shared by o12, o22, o33, o42
addDisjointConstraint(problem, "o12", "o22")
addDisjointConstraint(problem, "o12", "o33")
addDisjointConstraint(problem, "o12", "o42")
addDisjointConstraint(problem, "o22", "o33")
addDisjointConstraint(problem, "o22", "o42")
addDisjointConstraint(problem, "o33", "o42")

# R3 shared by o13, o31
addDisjointConstraint(problem, "o13", "o31")

addOrderConstraint(problem, "o11", "o12")
addOrderConstraint(problem, "o12", "o13")
addOrderConstraint(problem, "o21", "o22")
addOrderConstraint(problem, "o31", "o32")
addOrderConstraint(problem, "o32", "o33")
addOrderConstraint(problem, "o41", "o42")
 
print(problem.getSolutions()[0:4])

In [ ]:
print(problem.getSolutions()[0])

In [ ]:
def resource_for(jobs, task):
    for tasks in jobs:
        for t in tasks:
            if (t[0] == task):
                return t[1]
    return None

###

print("Task    Start Time    End Time   Uses")
print()
for task, bucket in sorted(problem.getSolutions()[0].items()):
    print("{:2}         {:2d}          {:2d}        {:2}".format(task, bucket*3, (bucket*3) + 2, resource_for(example, task)))